In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.2.2


In [2]:
import pickle
import pandas as pd


In [3]:
with open('../model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/Users/home/.local/share/virtualenvs/hw9-7bq1UQBo/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.2.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/Users/home/.local/share/virtualenvs/hw9-7bq1UQBo/lib/python3.8/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.2.2 when using version 1.1.3. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [6]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2022-02.parquet')

In [7]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,1,2022-02-01 00:06:58,2022-02-01 00:19:24,1.0,5.40,1.0,N,138,252,1,17.0,1.75,0.5,3.90,0.00,0.3,23.45,0.0,1.25,12.433333
1,1,2022-02-01 00:38:22,2022-02-01 00:55:55,1.0,6.40,1.0,N,138,41,2,21.0,1.75,0.5,0.00,6.55,0.3,30.10,0.0,1.25,17.550000
2,1,2022-02-01 00:03:20,2022-02-01 00:26:59,1.0,12.50,1.0,N,138,200,2,35.5,1.75,0.5,0.00,6.55,0.3,44.60,0.0,1.25,23.650000
3,2,2022-02-01 00:08:00,2022-02-01 00:28:05,1.0,9.88,1.0,N,239,200,2,28.0,0.50,0.5,0.00,3.00,0.3,34.80,2.5,0.00,20.083333
4,2,2022-02-01 00:06:48,2022-02-01 00:33:07,1.0,12.16,1.0,N,138,125,1,35.5,0.50,0.5,8.11,0.00,0.3,48.66,2.5,1.25,26.316667


In [8]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

Q1

What's the standard deviation of the predicted duration for this dataset?

In [10]:
y_pred.std()

5.28140357655334

Q2

Like in the course videos, we want to prepare the dataframe with the output.

First, let's create an artificial ride_id column:

df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

Next, write the ride id and the predictions to a dataframe with results.

Save it as parquet:
...

What's the size of the output file?

In [13]:
year = 2022
month = 2

In [14]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [16]:
df["y_pred"] = y_pred

In [17]:
df_result = df[['ride_id','y_pred']]

In [20]:
output_file = f"yellow_taxi_ride_{year}_{month:02d}.parquet"

In [21]:
output_file

'yellow_taxi_ride_2022_02.parquet'

In [30]:
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [31]:
!ls -lh $(output_file)

zsh:1: command not found: output_file
total 118824
-rw-r--r--  1 home  staff    14K Jun 25 20:39 starter.ipynb
-rw-r--r--  1 home  staff    57M Jun 25 20:42 yellow_taxi_ride_2022_02.parquet


Q3

Now let's turn the notebook into a script.

Which command you need to execute for that?

In [32]:
!jupyter nbconvert --to python starter.ipynb

[NbConvertApp] Converting notebook starter.ipynb to python
[NbConvertApp] Writing 1883 bytes to starter.py


Q5

Let's now make the script configurable via CLI. We'll create two parameters: year and month.

Run the script for March 2022.

What's the mean predicted duration?

Mean predicted duration 12.758556818790902